In [ ]:
import purly
from spectate import mvc
from example_utils import localhost
purly.state.Machine().daemon()

[2018-06-18 17:04:29 -0700] [21778] [INFO] Server Stopped


In [ ]:
websocket_url = localhost('ws', 8000) + "/example-model" + '/stream'

In [ ]:
layout = purly.Layout(websocket_url)
slider = layout.html('input', type='range', min=0, max=10, step=1, value=5)
layout.children.append(slider)

In [ ]:
layout

In [ ]:
@mvc.view(slider.attributes)
def printer(changes):
    for c in changes:
        print(c)

In [ ]:
layout.serve()